In [ ]:
from docplex.mp.model import Model
import networkx as nx
import numpy as np
import math as math
from time import time
from decimal import *
import random
import itertools
import matplotlib.pyplot as plt
import statistics
import warnings
warnings.filterwarnings("ignore")

In [37]:
##########-------------------->>>>>>>>>> TOPOLOGIES <<<<<<<<<<---------------------##########

##########----- Do not forget to modify the max number of switch ports/device -----########## 

N = {1,2,3,4,5,6}

E = {(1,2),(2,1),(2,3),(3,2),(3,4),(4,3),(4,5),(5,4),(2,5),(5,2),(4,6),(6,4)}

l = {(1,2):80,(2,1):80,(2,3):32,(3,2):32,(3,4):16,(4,3):16,(4,5):6,(5,4):6,(2,5):19,(5,2):19,(4,6):70,(6,4):70}

#max_value = max(l.values())

#for key in l:
#    l[key] = int((l[key]/max_value)*80)


##########-------------------->>>>>>>>>> ---------- <<<<<<<<<<--------------------##########

demand = [10]

avg_RU = []
cost_r = []
result_time = []

L = 256 
T = 1800
ts = 120
multi = 1

Ctx = 100
Crx = 150
Csxp = 10
Cch = 0.1

# Define the parameters
q = 0.5
alpha = 0.2  # dB/km
miu = 0.48
v = 0.05
Y_0 = 6.8 * 10 ** -6  # Per pulse
e_d = 0.023
e_0 = 0.5
fec = 1.22
n_tx = 0.1  # Transmittance in rx's si

# Create an empty list to store SKR values for each Ls
n_0, n_1, n_2 = {}, {}, {}
c_0, c_1, c_2 = {}, {}, {}
Q_miu_0, Q_miu_1, Q_miu_2 = {}, {}, {}
Q_v_0, Q_v_1, Q_v_2 = {}, {}, {}
E_miu_0, E_miu_1, E_miu_2 = {}, {}, {}
Q_1L_0, Q_1L_1, Q_1L_2 = {}, {}, {}
e_1U_0, e_1U_1, e_1U_2 = {}, {}, {}
H_2_E_miu_0, H_2_E_miu_1, H_2_E_miu_2 = {}, {}, {}
H_2_e_1U_0, H_2_e_1U_1, H_2_e_1U_2 = {}, {}, {}


SKR_0, SKR_1, SKR_2 = {}, {}, {}

for (i,j) in E:
    n_0[(i,j)] = n_tx * np.power(10, -1 * ((alpha * l[(i,j)] + 0) / 10))
    n_1[(i,j)] = n_tx * np.power(10, -1 * ((alpha * l[(i,j)] + 1.2) / 10))
    n_2[(i,j)] = n_tx * np.power(10, -1 * ((alpha * l[(i,j)] + 2.4) / 10))
    
    Q_miu_0[(i,j)] = Y_0 + 1 - np.exp(-1 * n_0[(i,j)] * miu) 
    Q_miu_1[(i,j)] = Y_0 + 1 - np.exp(-1 * n_1[(i,j)] * miu) 
    Q_miu_2[(i,j)] = Y_0 + 1 - np.exp(-1 * n_2[(i,j)] * miu) 
    
    Q_v_0[(i,j)] = Y_0 + 1 - np.exp(-1 * n_0[(i,j)] * v) 
    Q_v_1[(i,j)] = Y_0 + 1 - np.exp(-1 * n_1[(i,j)] * v) 
    Q_v_2[(i,j)] = Y_0 + 1 - np.exp(-1 * n_2[(i,j)] * v) 
    
    E_miu_0[(i,j)] = (e_0 * Y_0 + e_d * (1 - np.exp(-1 * n_0[(i,j)] * miu))) / Q_miu_0[(i,j)]  # Corrected this line
    E_miu_1[(i,j)] = (e_0 * Y_0 + e_d * (1 - np.exp(-1 * n_1[(i,j)] * miu))) / Q_miu_1[(i,j)]  # Corrected this line
    E_miu_2[(i,j)] = (e_0 * Y_0 + e_d * (1 - np.exp(-1 * n_2[(i,j)] * miu))) / Q_miu_2[(i,j)]  # Corrected this line
    
    Q_1L_0[(i,j)] = ((np.power(miu, 2) * np.exp(-1 * miu)) / (miu * v - np.power(v, 2))) * (
            Q_v_0[(i,j)] * np.exp(v) - Q_miu_0[(i,j)] * np.exp(miu) * (np.power(v, 2) / np.power(miu, 2)) - E_miu_0[(i,j)] * Q_miu_0[(i,j)] * np.exp(miu) * ((np.power(miu, 2) - np.power(v, 2)) / (np.power(miu, 2) * 0.5)))
    Q_1L_1[(i,j)] = ((np.power(miu, 2) * np.exp(-1 * miu)) / (miu * v - np.power(v, 2))) * (
            Q_v_1[(i,j)] * np.exp(v) - Q_miu_1[(i,j)] * np.exp(miu) * (np.power(v, 2) / np.power(miu, 2)) - E_miu_1[(i,j)] * Q_miu_1[(i,j)] * np.exp(miu) * ((np.power(miu, 2) - np.power(v, 2)) / (np.power(miu, 2) * 0.5)))
    Q_1L_2[(i,j)] = ((np.power(miu, 2) * np.exp(-1 * miu)) / (miu * v - np.power(v, 2))) * (
            Q_v_2[(i,j)] * np.exp(v) - Q_miu_2[(i,j)] * np.exp(miu) * (np.power(v, 2) / np.power(miu, 2)) - E_miu_2[(i,j)] * Q_miu_2[(i,j)] * np.exp(miu) * ((np.power(miu, 2) - np.power(v, 2)) / (np.power(miu, 2) * 0.5)))

    e_1U_0[(i,j)] = E_miu_0[(i,j)] * Q_miu_0[(i,j)]/Q_1L_0[(i,j)]  # Corrected this line
    e_1U_1[(i,j)] = E_miu_1[(i,j)] * Q_miu_1[(i,j)]/Q_1L_1[(i,j)]  # Corrected this line
    e_1U_2[(i,j)] = E_miu_2[(i,j)] * Q_miu_2[(i,j)]/Q_1L_2[(i,j)]  # Corrected this line
    
    H_2_E_miu_0[(i,j)] = (-1) * E_miu_0[(i,j)]/100 * np.log2(E_miu_0[(i,j)]/100) - (1 - E_miu_0[(i,j)]/100) * np.log2(1 - E_miu_0[(i,j)]/100)
    H_2_E_miu_1[(i,j)] = (-1) * E_miu_1[(i,j)]/100 * np.log2(E_miu_1[(i,j)]/100) - (1 - E_miu_1[(i,j)]/100) * np.log2(1 - E_miu_1[(i,j)]/100)
    H_2_E_miu_2[(i,j)] = (-1) * E_miu_2[(i,j)]/100 * np.log2(E_miu_2[(i,j)]/100) - (1 - E_miu_2[(i,j)]/100) * np.log2(1 - E_miu_2[(i,j)]/100)
    
    H_2_e_1U_0[(i,j)] = (-1) * e_1U_0[(i,j)] * np.log2(e_1U_0[(i,j)]) - (1 - e_1U_0[(i,j)]) * np.log2(1 - e_1U_0[(i,j)])
    H_2_e_1U_1[(i,j)] = (-1) * e_1U_1[(i,j)] * np.log2(e_1U_1[(i,j)]) - (1 - e_1U_1[(i,j)]) * np.log2(1 - e_1U_1[(i,j)])
    H_2_e_1U_2[(i,j)] = (-1) * e_1U_2[(i,j)] * np.log2(e_1U_2[(i,j)]) - (1 - e_1U_2[(i,j)]) * np.log2(1 - e_1U_2[(i,j)])

    SKR_0[(i,j)] = ((q * (Q_1L_0[(i,j)] * (1 - H_2_e_1U_0[(i,j)]) - Q_miu_0[(i,j)] * fec * H_2_E_miu_0[(i,j)])) * 2.5 * np.power(10, 9))/3 # Corrected this line
    SKR_1[(i,j)] = ((q * (Q_1L_1[(i,j)] * (1 - H_2_e_1U_1[(i,j)]) - Q_miu_1[(i,j)] * fec * H_2_E_miu_1[(i,j)])) * 2.5 * np.power(10, 9))/3 # Corrected this line
    SKR_2[(i,j)] = ((q * (Q_1L_2[(i,j)] * (1 - H_2_e_1U_2[(i,j)]) - Q_miu_2[(i,j)] * fec * H_2_E_miu_2[(i,j)])) * 2.5 * np.power(10, 9))/3 # Corrected this line
    
#Topology
G = nx.DiGraph()
G.add_nodes_from(N)
G.add_edges_from(E)

c1 = set(itertools.combinations(N, 2))

def path_length(p):
    pl = sum(l[(i,j)] for (i,j) in p)
    return pl

def path_std_dev(p):
    my_array = [l[(i,j)] for (i,j) in p]
    pl = statistics.stdev(my_array) if len(my_array) > 1 else 0
    return pl

#All of paths for each request
all_paths = {}
for r in c1:
    paths = list(nx.all_simple_edge_paths(G, source=r[0], target=r[1]))
    sorted_paths = sorted(paths, key=lambda x: (len(x), path_length(x)))
    sorted_paths = sorted(paths, key=lambda x: (len(x), path_length(x), path_std_dev(x)))
    all_paths[r] = [p for p in sorted_paths if len(p) - len(sorted_paths[0]) == 0]

#Set aux for decision variable path 
aux_paths = []
for (s,d) in c1:
    for path in all_paths[(s,d)]:
        aux_paths.append(((s,d),tuple(path)))

X = [1,2,3,4,5]
aux_x = []
for n in N:
    for x in X:
        aux_x.append((n,x))

Y = [1,2,3,4,5]
aux_y = []
for n in N:
    for y in Y:
        aux_y.append((n,y))
        
Z = [1,2,3,4,5]
S = [1,2,3]   #max number of ports of a switch
aux_z = []
for n in N:
    for z in Z:
        aux_z.append((n,z))
        
aux_xx = []
for n in N:
    for x in X:
        for x_p in X:
            for (i,j) in E:
                if i == n:
                    aux_xx.append(((i,x),(j,x_p)))
                    
aux_yz = []
for n in N:
    for y in Y:
        for z in Z:
            for s in S:
                for (i,j) in E:
                    if i == n:
                        aux_yz.append(((i,y),(j,z,s)))
                        aux_yz.append(((i,z,s),(j,y)))
                        
aux_zz = []
for n in N:
    for z in Z:
        for z_p in Z:
            for s in S:
                for s_p in S:
                    for (i,j) in E:
                        if i == n:
                            aux_zz.append(((i,z,s),(j,z_p,s_p)))


In [38]:
for dem in demand:
    R = {}
    h = {}
    for (i,j) in c1:
        R[(i,j)] = dem

    #R[(2,5)] = 200
    #R[(4,5)] = 400
        
    time_0 = time()
    
    m = Model(name='qkd_network_with_TDM')
    
    ##### Decision variable #####
    path = m.binary_var_dict(aux_paths, name="path")
    
    t_0 = m.binary_var_dict(aux_x, name="t_0")
    r_0 = m.binary_var_dict(aux_x, name="r_0")
    
    t_1 = m.binary_var_dict(aux_y, name="t_1")
    r_1 = m.binary_var_dict(aux_y, name="r_1")
    
    t_2 = m.binary_var_dict(aux_z, name="t_2")
    r_2 = m.binary_var_dict(aux_z, name="r_2")
    
    ch_0 = m.binary_var_dict(aux_xx, name="ch_0")
    ch_1 = m.binary_var_dict(aux_yz, name="ch_1")
    ch_2 = m.binary_var_dict(aux_zz, name="ch_2")
    
    slack_0 = m.continuous_var_dict(aux_xx, name="slack_0")
    slack_1 = m.continuous_var_dict(aux_yz, name="slack_1")
    slack_2 = m.continuous_var_dict(aux_zz, name="slack_2")
    
    ### Auxiliary definitions #####
    h = {(i,j): m.sum(m.sum(R[(s,d)]*path[((s,d),tuple(p))] for p in all_paths[(s,d)] if (i,j) in p or (j,i) in p) for (s,d) in R) for (i,j) in E}

    Tx_n = {n: m.sum(t_0[(n,x)] for x in X) + m.sum(t_1[(n,y)] for y in Y) + m.sum(t_2[(n,z)] for z in Z) for n in N}

    Rx_n = {n: m.sum(r_0[(n,x)] for x in X) + m.sum(r_1[(n,y)] for y in Y) + m.sum(r_2[(n,z)] for z in Z) for n in N}
     
    q_ch = {(i,j): (m.sum(m.sum(ch_0[((i,x),(j,x_p))] for x_p in X) for x in X) +
                    m.sum(m.sum(m.sum(ch_1[((j,y),(i,z,s))] + ch_1[((i,z,s),(j,y))] for s in S) for z in Z) for y in Y) +
                    m.sum(m.sum(m.sum(m.sum(ch_2[((i,z,s),(j,z_p,s_p))] for s_p in S) for z_p in Z) for s in S) for z in Z)) for (i,j) in E}
    
    Sx_n = {n: m.sum(m.sum(m.sum(m.sum(ch_1[((i,z,s),(j,y))] + ch_1[((j,y),(i,z,s))] for y in Y) for s in S) for z in Z) for (i,j) in E if i == n) +
               m.sum(m.sum(m.sum(m.sum(m.sum(ch_2[((i,z,s),(j,z_p,s_p))] + ch_2[((j,z_p,s_p),(i,z,s))] for s_p in S) for z_p in Z) for s in S) for z in Z) for (i,j) in E if i == n) +
               m.sum(r_2[(n,z)] for z in Z) +
               m.sum(t_2[(n,z)] for z in Z) for n in N}
    
    ##### Objective function #####
    m.minimize(m.sum(Ctx*Tx_n[n] for n in N) + 
               m.sum(Crx*Rx_n[n] for n in N) + 
               m.sum(Cch*l[(i,j)]*q_ch[(i,j)] for (i,j) in E) +
               m.sum(Csxp*Sx_n[n] for n in N))
    
    ##### Constraints #####
    
    # 1- Ensuring select a path for each request
    for (s,d) in R:
        if R[(s,d)] != 0:
            m.add_constraint(multi - m.sum(path[((s,d),tuple(p))] for p in all_paths[(s,d)]) <= 0)   

    # 2- A device or switch port can be use just once
    for n in N:
        for x in X:
            m.add_constraint(m.sum(m.sum(ch_0[((i,x),(j,x_p))] for x_p in X) for (i,j) in E if i == n) <= 1)
            m.add_constraint(m.sum(m.sum(ch_0[((i,x_p),(j,x))] for x_p in X) for (i,j) in E if j == n) <= 1)
    
    for n in N:
        for y in Y:
            m.add_constraint(m.sum(m.sum(m.sum(ch_1[((i,y),(j,z,s))] for s in S) for z in Z) for (i,j) in E if i == n) <= 1)
            m.add_constraint(m.sum(m.sum(m.sum(ch_1[((i,z,s),(j,y))] for s in S) for z in Z) for (i,j) in E if j == n) <= 1)
    
    for n in N:
        for z in Z:
            for s in S:
                m.add_constraint(m.sum(m.sum(ch_1[((i,z,s),(j,y))] for y in Y) for (i,j) in E if i == n) +
                                 m.sum(m.sum(m.sum(ch_2[((i,z,s),(j,z_p,s_p))] for s_p in S) for z_p in Z) for (i,j) in E if i == n) <= 1)
                    
                m.add_constraint(m.sum(m.sum(ch_1[((i,y),(j,z,s))] for y in Y) for (i,j) in E if j == n) +
                                 m.sum(m.sum(m.sum(ch_2[((i,z_p,s_p),(j,z,s))] for s_p in S) for z_p in Z) for (i,j) in E if j == n) <= 1)
    
    # 3- Calculate the Tx and Rx per nodes
    for (i,j) in E:
        for x in X:
            for x_p in X:
                m.add_constraint(2*ch_0[((i,x),(j,x_p))] - t_0[(i,x)] - r_0[(j,x_p)] <= 0)
    
    for (i,j) in E:
        for y in Y:
            for z in Z:
                for s in S:
                    m.add_constraint(2*ch_1[((j,y),(i,z,s))] - t_1[(j,y)] - r_2[(i,z)] <= 0)
                    m.add_constraint(2*ch_1[((i,z,s),(j,y))] - t_2[(i,z)] - r_1[(j,y)] <= 0)     
    
    for (i,j) in E:
        for z in Z:
            for s in S:
                for z_p in Z:
                    for s_p in S:
                        m.add_constraint(2*ch_2[((i,z,s),(j,z_p,s_p))] - t_2[(i,z)] - r_2[(j,z_p)] <= 0)    
    
    # 4- Relation between the channels and the slacks variables
    for (i,j) in E:
        for x in X:
            for x_p in X:
                m.add_constraint(slack_0[((i,x),(j,x_p))] - T*ch_0[((i,x),(j,x_p))] <= 0)
    
    for (i,j) in E:
        for y in Y:
            for z in Z:
                for s in S:
                    m.add_constraint(slack_1[((j,y),(i,z,s))] - (T-ts)*ch_1[((j,y),(i,z,s))] <= 0)
                    m.add_constraint(slack_1[((i,z,s),(j,y))] - (T-ts)*ch_1[((i,z,s),(j,y))] <= 0)
                        
    for (i,j) in E:
        for z in Z:
            for s in S:
                for z_p in Z:
                    for s_p in S:
                        m.add_constraint(slack_2[((i,z,s),(j,z_p,s_p))] - (T-ts)*ch_2[((i,z,s),(j,z_p,s_p))] <= 0)

    # 5- The sum of the slack for a devices has to be <= T
    for n in N:
        for z in Z:
            m.add_constraint(m.sum(m.sum(m.sum((slack_1[((i,z,s),(j,y))] + ts*ch_1[((i,z,s),(j,y))]) for y in Y) for s in S) for (i,j) in E if i == n) +
                             m.sum(m.sum(m.sum(m.sum((slack_2[((i,z,s),(j,z_p,s_p))] + ts*ch_2[((i,z,s),(j,z_p,s_p))]) for s_p in S) for z_p in Z) for s in S) for (i,j) in E if i == n) - T <= 0)
                
    for n in N:
        for z in Z:
            m.add_constraint(m.sum(m.sum(m.sum((slack_1[((i,y),(j,z,s))] + ts*ch_1[((i,y),(j,z,s))]) for y in Y) for s in S) for (i,j) in E if j == n) +
                             m.sum(m.sum(m.sum(m.sum((slack_2[((i,z_p,s_p),(j,z,s))] + ch_2[((i,z_p,s_p),(j,z,s))]*ts) for s_p in S) for z_p in Z) for s in S) for (i,j) in E if j == n) - T <= 0)
    
    # 6- Calculating the number of resource to meet the demands h[(i,j)]
    for (i,j) in E:
        m.add_constraint(((SKR_0[(i,j)]*m.sum(m.sum(slack_0[((i,x),(j,x_p))] + slack_0[((j,x),(i,x_p))] for x_p in X) for x in X) + 
                           SKR_1[(i,j)]*m.sum(m.sum(m.sum(slack_1[((i,y),(j,z,s))] + slack_1[((j,y),(i,z,s))] + slack_1[((i,z,s),(j,y))] + slack_1[((j,z,s),(i,y))] for s in S) for z in Z) for y in Y)  +
                           SKR_2[(i,j)]*m.sum(m.sum(m.sum(m.sum(slack_2[((i,z,s),(j,z_p,s_p))] + slack_2[((j,z,s),(i,z_p,s_p))] for s_p in S) for z_p in Z) for s in S) for z in Z) )/(T*L)) == h[(i,j)])
           
    ##### Running the model and getting results #####
    #print(m.export_to_string())
    sol = m.solve(log_output=False)
    sol.display() 
    

solution for: qkd_network_with_TDM
objective: 1400.300
status: OPTIMAL_SOLUTION(2)
path_(2, 4)_((2, 5), (5, 4)) = 1
path_(1, 2)_((1, 2),) = 1
path_(3, 4)_((3, 4),) = 1
path_(1, 5)_((1, 2), (2, 5)) = 1
path_(4, 6)_((4, 6),) = 1
path_(1, 4)_((1, 2), (2, 5), (5, 4)) = 1
path_(2, 3)_((2, 3),) = 1
path_(4, 5)_((4, 5),) = 1
path_(2, 6)_((2, 5), (5, 4), (4, 6)) = 1
path_(5, 6)_((5, 4), (4, 6)) = 1
path_(3, 6)_((3, 4), (4, 6)) = 1
path_(1, 6)_((1, 2), (2, 3), (3, 4), (4, 6)) = 1
path_(2, 5)_((2, 5),) = 1
path_(1, 3)_((1, 2), (2, 3)) = 1
path_(3, 5)_((3, 4), (4, 5)) = 1
t_0_2_2 = 1
t_0_2_3 = 1
t_0_6_4 = 1
r_0_1_1 = 1
r_0_1_2 = 1
r_0_4_3 = 1
t_2_3_3 = 1
t_2_5_5 = 1
r_2_2_4 = 1
r_2_4_1 = 1
ch_0_(2, 2)_(1, 2) = 1
ch_0_(2, 3)_(1, 1) = 1
ch_0_(6, 4)_(4, 3) = 1
ch_2_(3, 3, 3)_(4, 1, 2) = 1
ch_2_(3, 3, 2)_(2, 4, 3) = 1
ch_2_(5, 5, 3)_(4, 1, 1) = 1
ch_2_(5, 5, 1)_(2, 4, 1) = 1
slack_0_(2, 2)_(1, 2) = 1418.296
slack_0_(2, 3)_(1, 1) = 1800.000
slack_0_(6, 4)_(4, 3) = 1379.796
slack_2_(3, 3, 3)_(4, 1, 2) 

In [39]:
time_f = time()
print(round(time_f - time_0,2))

46.07


In [40]:
dict_Tx_n = {n: round(Tx_n[n].solution_value) for n in N}
dict_Rx_n = {n: round(Rx_n[n].solution_value) for n in N}
dict_Sx_n = {n: round(Sx_n[n].solution_value) for n in N}
dict_q_ch = {(i,j): round(q_ch[(i,j)].solution_value) for (i,j) in E}
dict_h = {(i,j): round(h[(i,j)].solution_value) for (i,j) in E}
print(f"dict_Tx_n = {dict_Tx_n}")
print(f"dict_Rx_n = {dict_Rx_n}")
print(f"dict_Sx_n = {dict_Sx_n}")
print(f"dict_q_ch = {dict_q_ch}")
print(f"dict_h = {dict_h}")

dict_Tx_n = {1: 0, 2: 2, 3: 1, 4: 0, 5: 1, 6: 1}
dict_Rx_n = {1: 2, 2: 1, 3: 0, 4: 2, 5: 0, 6: 0}
dict_Sx_n = {1: 0, 2: 3, 3: 3, 4: 3, 5: 3, 6: 0}
dict_q_ch = {(1, 2): 0, (2, 1): 2, (3, 4): 1, (4, 3): 0, (5, 4): 1, (4, 6): 0, (6, 4): 1, (2, 3): 0, (4, 5): 0, (3, 2): 1, (2, 5): 0, (5, 2): 1}
dict_h = {(1, 2): 50, (2, 1): 50, (3, 4): 40, (4, 3): 40, (5, 4): 60, (4, 6): 50, (6, 4): 50, (2, 3): 30, (4, 5): 60, (3, 2): 30, (2, 5): 50, (5, 2): 50}


In [41]:
##### ----- Resource Utilization ----- #####
F_T, F_R, F_T_p, F_R_p, F_T_p_p, F_R_p_p = 0, 0, 0, 0, 0, 0  

F_T = (1/T)*sum(sum(sum(sum(slack_0[((i,x),(j,x_p))].solution_value for x_p in X) for (i,j) in E if i == n) for x in X) for n in N)
F_R = (1/T)*sum(sum(sum(sum(slack_0[((i,x),(j,x_p))].solution_value for x in X) for (i,j) in E if j == n) for x_p in X) for n in N)

F_T_p = (1/T)*sum(sum(sum(sum(sum(slack_1[((i,y),(j,z,s))].solution_value for s in S) for z in Z) for (i,j) in E if i == n) for y in Y) for n in N)
F_R_p = (1/T)*sum(sum(sum(sum(sum(slack_1[((i,z,s),(j,y))].solution_value for s in S) for z in Z) for (i,j) in E if j == n) for y in Y) for n in N)

F_T_p_p = (1/T)*(sum(sum(sum(sum(sum(slack_1[((i,z,s),(j,y))].solution_value for y in Y) for s in S) for (i,j) in E if i == n) for z in Z) for n in N) + 
                 sum(sum(sum(sum(sum(sum(slack_2[((i,z,s),(j,z_p,s_p))].solution_value for s_p in S) for z_p in Z) for s in S) for (i,j) in E if i == n) for z in Z) for n in N)) 

F_R_p_p = (1/T)*(sum(sum(sum(sum(sum(slack_1[((i,y),(j,z,s))].solution_value for y in Y) for s in S) for (i,j) in E if j == n) for z in Z) for n in N) + 
                 sum(sum(sum(sum(sum(sum(slack_2[((i,z_p,s_p),(j,z,s))].solution_value for s_p in S) for z_p in Z) for s in S) for (i,j) in E if i == n) for z in Z) for n in N)) 

RU_mean =  ((F_T + F_R + F_T_p + F_R_p + F_T_p_p + F_R_p_p)/sum((Tx_n[n].solution_value + Rx_n[n].solution_value) for n in N))*100

print(F_T)
print(F_R)
print(F_T_p)
print(F_R_p)
print(F_T_p_p)
print(F_R_p_p)
print(RU_mean)


2.554495514420124
2.554495514420124
0.0
0.0
0.2782756158981485
0.2782756158981485
56.65542260636545
